In [ ]:
!pip install faiss
!pip install sentence-transformers
!sudo apt-get install libomp-dev

In [ ]:
titles = [title.replace("\n", "").replace("How to", "").replace("- wikiHow", "").strip().replace("İ", "i").replace("I", "ı").lower() for title in open("/content/drive/MyDrive/necessities/data/titles_tr.txt", "r", encoding="utf-8").readlines()]

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sts-pseudo_nli-dbmdz-convbert-base-turkish-mc4-uncased")

title_embeddings = model.encode(titles)
print("Title embeddings tamamlandı.")

In [ ]:
import faiss

d = title_embeddings.shape[1]

index = faiss.IndexFlatL2(d)
index.add(title_embeddings)
index.ntotal

In [ ]:
import json
retrieval_dict = {}
step2goal = json.load(open("/content/drive/MyDrive/necessities/data/step2goal.json", "r", encoding="utf-8"))

In [ ]:
print(len(step2goal.keys()))

In [ ]:
for step in list(step2goal.keys()):
  print(list(step2goal.keys()).index(step))
  step_embedding = model.encode(step.replace("İ", "i").replace("I", "ı").lower()).reshape((1, title_embeddings.shape[1]))
  corresponding_goal = step2goal[step]
  gold_goal = None
  
  k = 30
  D, I = index.search(step_embedding, k)

  retrieved_goals = []
  retrieved_goals_similarity = []
  for i in range(len(I[0])):
    retrieved_goal_embedding = model.encode(titles[I[0][i]])
    cosine_scores = util.cos_sim(step_embedding, retrieved_goal_embedding)

    goal = titles[I[0][i]]
    if goal[0] == "i":
      goal = "İ" + goal[1:]

    elif goal[0] == "ı":
      goal = "I" + goal[1:]

    else:
      goal = goal.capitalize()


    retrieved_goals.append(goal)
    retrieved_goals_similarity.append(float(cosine_scores[0][0]))
  
  retrieved_description_similarity = []
  weighted_sum_similarity = []
  retrieved_goal_rank = -1

  retrieval_dict[step] = {"corresponding_goal":corresponding_goal, "gold_goal":gold_goal, "retrieved_goals":retrieved_goals, "retrieved_goals_similarity":retrieved_goals_similarity, "retrieved_description_similarity":retrieved_description_similarity, "weighted_sum_similarity":weighted_sum_similarity, "retrieved_goal_rank":retrieved_goal_rank}

In [ ]:
with open("tüm_wikihow_e30.json", "w", encoding="utf-8") as outfile:
    json.dump(retrieval_dict, outfile, indent=3, ensure_ascii=False)